In [3]:
import os
import pandas as pd
import numpy as np
import itertools
from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn import metrics
import scipy

C:\Users\SungJoonPark\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
sparse_matrix = scipy.sparse.load_npz('../data/TCGA/npz/TCGA_nofiltering.npz')
feature_array = np.array(pd.read_csv("../data/TCGA/npz/features.txt",header=None)[0].tolist())
print(sparse_matrix.shape)
feature_index = np.where(sparse_matrix.sum(axis=0)>4)[1]
sparse_matrix_temp = sparse_matrix[:,feature_index]
print(sparse_matrix_temp.shape)
sample_index = np.where(sparse_matrix_temp.sum(axis=1)!=0)[0]
sparse_matrix_temp = sparse_matrix_temp[sample_index,:]
print(sparse_matrix_temp.shape)


X = sparse_matrix_temp

feature_array = feature_array[feature_index]
print(feature_array.shape)



label_array = np.array(pd.read_csv("../data/TCGA/npz/labels.txt",header=None)[0].tolist())
label_array = label_array[sample_index]
le = preprocessing.LabelEncoder()
le.fit(label_array)
y = le.transform(label_array)
print(y.shape)

X, y = shuffle(X, y, random_state=0)

(9822, 1700753)
(9822, 5696)
(8342, 5696)
(5696L,)
(8342L,)


In [23]:
class_weight = 'balanced'
n_tree = 500

result_dict={}
for cancer in list(set(label_array)):
# for cancer in ['nsclc']:
    
    print(cancer)
    y_temp = np.copy(y)+1
    class_ = le.transform([cancer])[0]+1
    print class_
    y_temp[y_temp!=class_]=0
    y_temp[y_temp==class_]=1

    
    over_mut_list = []
    under_mut_list = []
    for mut_index in range(len(feature_array)):
        over_score = (X[np.where(y_temp==1)[0],mut_index].sum()) / float(len(np.where(y_temp==1)[0]))
        under_score = (X[np.where(y_temp==0)[0],mut_index].sum()) / float(len(np.where(y_temp==0)[0]))
        if over_score>under_score:
            over_mut_list.append(feature_array[mut_index]) 
        elif over_score<under_score:
            under_mut_list.append(feature_array[mut_index]) 
        else:
            raise ValueError("over_score is same as under score")
    
    skf = StratifiedKFold(y,n_folds=10,shuffle=False)
    cv_pred_score_list = []
    cv_pred_binary_list=[]
    cv_test_list=[]
    
    
    print np.unique(y_temp,return_counts=True)
    positive_class_num = float(len(np.where(y_temp==1)[0]))
    
    for train_index, test_index in skf:
        X_train, X_test = X[train_index,:], X[test_index,:]
        y_train, y_test = y_temp[train_index], y_temp[test_index]
        
        
        clf = RandomForestClassifier(random_state=0, n_estimators=n_tree ,class_weight='balanced')

        clf.fit(X_train,y_train)
        
        pred_score_list = clf.predict_proba(X_test)[:,1].tolist()
        pred_binary_list = clf.predict(X_test).tolist()
        cv_pred_binary_list.append(pred_binary_list)
        cv_pred_score_list.append(pred_score_list)
        cv_test_list.append(y_test.tolist())
    

    #Average Score
    auroc_kfold_list = []
    precision_kfold_list = []
    recall_kfold_list = []
    f1_kfold_list = []
    
    for k in range(len(cv_test_list)):
        cv_test_list_kfold = cv_test_list[k]
        cv_pred_score_list_kfold = cv_pred_score_list[k]
        cv_pred_binary_list_kfold=cv_pred_binary_list[k]
        
        auroc_kfold = metrics.roc_auc_score(cv_test_list_kfold,cv_pred_score_list_kfold)
        precision_kfold = metrics.precision_score(cv_test_list_kfold,cv_pred_binary_list_kfold)
        recall_kfold = metrics.recall_score(cv_test_list_kfold,cv_pred_binary_list_kfold)
        f1_kfold = metrics.f1_score(cv_test_list_kfold,cv_pred_binary_list_kfold)
        
        auroc_kfold_list.append(auroc_kfold)
        precision_kfold_list.append(precision_kfold)
        recall_kfold_list.append(recall_kfold)
        f1_kfold_list.append(f1_kfold)
        
    auroc_average_score = np.mean(auroc_kfold_list)
    precision_average_score = np.mean(precision_kfold_list)
    recall_average_score = np.mean(recall_kfold_list)
    f1_average_score = np.mean(f1_kfold_list)
        
    
    #Flatten List Calculation
    cv_pred_score_flatten_list = list(itertools.chain(*cv_pred_score_list))
    cv_pred_binary_flatten_list = list(itertools.chain(*cv_pred_binary_list))
    cv_test_flatten_list = list(itertools.chain(*cv_test_list))
    fpr, tpr, thresholds  = metrics.roc_curve(cv_test_flatten_list, cv_pred_score_flatten_list, drop_intermediate=False)
    auroc = metrics.auc(fpr,tpr)
    precision = metrics.precision_score(cv_test_flatten_list,cv_pred_binary_flatten_list)
    recall = metrics.recall_score(cv_test_flatten_list,cv_pred_binary_flatten_list)
    f1_score = metrics.f1_score(cv_test_flatten_list,cv_pred_binary_flatten_list)

    
    
    if class_weight =='balanced':
        clf = RandomForestClassifier(random_state=0, n_estimators=n_tree ,class_weight='balanced')
    else:
        clf = RandomForestClassifier(random_state=0, n_estimators=n_tree)

    clf.fit(X,y_temp)
    fis = pd.Series(clf.feature_importances_, index=feature_array)
    fis = fis.sort_values(ascending=False)
    fis = fis[fis>0]
    fis_sorted_mut = fis.index.tolist()
    
    fis_over = fis[over_mut_list].sort_values(ascending=False)
    fis_under = fis[under_mut_list].sort_values(ascending=False)
    fis_over_sorted_mut = fis_over.index.tolist()
    fis_under_sorted_mut = fis_under.index.tolist()
    
    fis_over_sorted_genes = []
    for mut in fis_over.index:
        gene = mut.split(":")[0]
        if gene not in fis_over_sorted_genes:
            fis_over_sorted_genes.append(gene)

    fis_under_sorted_genes = []
    for mut in fis_under.index:
        gene = mut.split(":")[0]
        if gene not in fis_under_sorted_genes:
            fis_under_sorted_genes.append(gene)

    result_list = [cancer.upper(),positive_class_num,auroc_average_score,auroc,fis_over_sorted_mut[0:100]]

    
    result_dict[cancer]=result_list
    

esca
8
(array([0, 1], dtype=int64), array([8161,  181], dtype=int64))
dlbc
7
(array([0, 1], dtype=int64), array([8297,   45], dtype=int64))
ucs
30
(array([0, 1], dtype=int64), array([8285,   57], dtype=int64))
thym
28
(array([0, 1], dtype=int64), array([8273,   69], dtype=int64))
pcpg
21
(array([0, 1], dtype=int64), array([8179,  163], dtype=int64))
blca
2
(array([0, 1], dtype=int64), array([8214,  128], dtype=int64))
ucec
29
(array([0, 1], dtype=int64), array([8096,  246], dtype=int64))
uvm
31
(array([0, 1], dtype=int64), array([8269,   73], dtype=int64))
cesc
4
(array([0, 1], dtype=int64), array([8191,  151], dtype=int64))
lgggbm
16
(array([0, 1], dtype=int64), array([7632,  710], dtype=int64))
sarc
23
(array([0, 1], dtype=int64), array([8224,  118], dtype=int64))
kirp
13
(array([0, 1], dtype=int64), array([8179,  163], dtype=int64))
laml
14
(array([0, 1], dtype=int64), array([8223,  119], dtype=int64))
coadread
6
(array([0, 1], dtype=int64), array([7901,  441], dtype=int64))
prad
22